## Task 2.2

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# Load MNIST data
def load_mnist():
    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    def preprocess(image, label):
        image = tf.cast(image, tf.float32)
        image = image / 255.0
        image = tf.reshape(image, [-1])
        return image, label

    ds_train = ds_train.map(preprocess).batch(128).prefetch(tf.data.experimental.AUTOTUNE)
    return ds_train

# Define the Generator and Discriminator networks
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(784, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x

# Logistic loss functions
def discriminator_loss(logits_real, logits_fake):
    loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_real, labels=tf.ones_like(logits_real)))
    loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake, labels=tf.zeros_like(logits_fake)))
    return loss_real + loss_fake

def generator_loss(logits_fake):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake, labels=tf.ones_like(logits_fake)))

# Training step
@tf.function
def train_step(generator, discriminator, images, batch_size, noise_dim, generator_optimizer, discriminator_optimizer):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        logits_real = discriminator(images, training=True)
        logits_fake = discriminator(generated_images, training=True)

        gen_loss = generator_loss(logits_fake)
        disc_loss = discriminator_loss(logits_real, logits_fake)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training loop
def train(generator, discriminator, dataset, epochs, batch_size, noise_dim, generator_optimizer, discriminator_optimizer):
    for epoch in range(epochs):
        for image_batch, _ in dataset:
            gen_loss, disc_loss = train_step(generator, discriminator, image_batch, batch_size, noise_dim, generator_optimizer, discriminator_optimizer)

        if epoch % 1000 == 0:
            print(f'Epoch {epoch}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}')
            generate_and_save_images(generator, epoch + 1, noise_dim)

def generate_and_save_images(model, epoch, noise_dim):
    noise = tf.random.normal([16, noise_dim])
    predictions = model(noise, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i].numpy().reshape(28, 28), cmap='gray')
        plt.axis('off')

    plt.savefig(f'out/image_at_epoch_{epoch:04d}.png')
    plt.close(fig)

# Main script
noise_dim = 100
batch_size = 128

generator = Generator()
discriminator = Discriminator()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

ds_train = load_mnist()

# Train for 20k iterations
epochs = 20000
train(generator, discriminator, ds_train, epochs, batch_size, noise_dim, generator_optimizer, discriminator_optimizer)

# Train for 100k iterations
epochs = 100000
train(generator, discriminator, ds_train, epochs, batch_size, noise_dim, generator_optimizer, discriminator_optimizer)


2024-05-26 16:22:19.532249: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 16:22:19.607730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-26 16:22:22.073515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task

Epoch 0, Generator Loss: 0.6923880577087402, Discriminator Loss: 1.0090402364730835


2024-05-26 16:22:34.441205: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 16:22:38.746890: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 16:22:43.127150: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 16:22:47.493572: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 16:22:51.918175: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 16:22:56.221157: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 16:23:00.361397: W tensorflow/core/framework/local_rendezvous.cc:404] L

Epoch 1000, Generator Loss: 0.6931471824645996, Discriminator Loss: 1.3862943649291992


2024-05-26 17:33:51.829766: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 17:33:56.135924: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 17:34:00.443823: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 17:34:04.634016: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 17:34:08.909641: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 17:34:13.088649: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-26 17:34:17.281613: W tensorflow/core/framework/local_rendezvous.cc:404] L

...

2024-05-27 18:50:54.698264: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 18:50:58.963434: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 18:51:03.289021: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 18:51:07.483250: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 18:51:11.669310: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 18:51:16.004941: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 18:51:20.329335: W tensorflow/core/framework/local_rendezvous.cc:404] L

Epoch 4000, Generator Loss: 0.6931471824645996, Discriminator Loss: 1.3862943649291992


2024-05-27 19:59:14.610477: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 19:59:18.227197: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 19:59:22.076189: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 19:59:25.620628: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 19:59:29.381315: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 19:59:33.189094: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 19:59:37.085291: W tensorflow/core/framework/local_rendezvous.cc:404] L

Epoch 5000, Generator Loss: 0.6931471824645996, Discriminator Loss: 1.3862943649291992


2024-05-27 21:05:53.283180: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 21:05:57.549633: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 21:06:01.797789: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 21:06:06.061130: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 21:06:10.377310: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 21:06:14.662633: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-27 21:06:18.904774: W tensorflow/core/framework/local_rendezvous.cc:404] L